In [1]:
import pandas as pd
import numpy as np
from datetime import date
import csv 
import requests
import glob
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
from util import  select_relevant_rows, select_attributes, read_movement_data, read_multiple_csv, download_updated_mobility_data

In [2]:
local_region_path = r'/Users/marcomazzoli/Documents/Projects/COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'
remote_region_path = r'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

regions_frame = pd.read_csv(remote_region_path)

In [3]:
region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'

region_focus_data = select_relevant_rows(
    regions_frame,
    attribute_focus,
    region_focus)

In [4]:
list(region_focus_data)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
frame_interesting_columns = select_attributes(region_focus_data, [
    'data',
    'ricoverati_con_sintomi',
    'terapia_intensiva',
    'totale_ospedalizzati',
    'variazione_totale_positivi',
    'nuovi_positivi','deceduti',
    'tamponi',
    'ingressi_terapia_intensiva'
    ])

In [6]:
frame_interesting_columns.head()

,data,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,variazione_totale_positivi,nuovi_positivi,deceduti,tamponi,ingressi_terapia_intensiva
4,2020-02-24T18:00:00,10,2,12,0,18,0,148,NaN
25,2020-02-25T18:00:00,15,2,17,8,8,0,391,NaN
46,2020-02-26T18:00:00,20,3,23,20,21,1,577,NaN
67,2020-02-27T18:00:00,36,6,42,50,50,1,1033,NaN
88,2020-02-28T18:00:00,56,6,62,47,48,2,1277,NaN


In [7]:
frame_interesting_columns = pd.DataFrame(frame_interesting_columns)
frame_interesting_columns['data'] = pd.to_datetime(frame_interesting_columns['data'])
frame_interesting_columns['data'] = frame_interesting_columns['data'].dt.strftime(r'%Y-%m-%d')


In [9]:
# 'tamponi' is cumulative, revert to incidence
columns_to_normalize = ['deceduti', 'tamponi']

for column in columns_to_normalize:
    frame_interesting_columns[column] = frame_interesting_columns[column].transform(
        lambda s: s.sub(s.shift().fillna(0)).abs()
        )


SyntaxError: invalid syntax (<ipython-input-9-d4666c2a54f6>, line 5)

In [ ]:
import requests
import os.path, time
from datetime import date
from util import download_updated_mobility_data

mobility_data_url = r'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'
file_path = r'/Users/marcomazzoli/Documents/Scuola/Global_Mobility_Report.csv'
mobility_data_zip_url = r'https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip'
zip_path = r'/Users/marcomazzoli/Documents/Scuola/Region_Mobility_Report_CSVs.zip'
region_mobility_path = r'/Users/marcomazzoli/Documents/Scuola/Region_Mobility_Report_CSVs'

download_updated_mobility_data(mobility_data_url, file_path)

mobility_df = read_movement_data(
    region_mobility_path,
    'IT_Region_Mobility_Report',
    region='Emilia-Romagna')


In [ ]:
list(mobility_df)

In [ ]:
mobility_df = mobility_df[['date',
            'retail_and_recreation_percent_change_from_baseline',
            'grocery_and_pharmacy_percent_change_from_baseline',
            'parks_percent_change_from_baseline',
            'transit_stations_percent_change_from_baseline',
            'workplaces_percent_change_from_baseline',
            'residential_percent_change_from_baseline']]

In [ ]:
mobility_df.head()

In [ ]:
frame_interesting_columns.rename(columns={'data': 'date'}, inplace=True)
frame_interesting_columns.set_index('date',inplace=True)

In [ ]:
mobility_df.set_index('date',inplace=True)

In [ ]:
merged = pd.merge(
    frame_interesting_columns,
    mobility_df,
    on='date'
    )
merged = merged.fillna(0)

In [ ]:
list(merged)

In [ ]:
merged.tail()

In [ ]:
dataset = merged
values = merged.values

groups = range(8)
i = 1
pyplot.figure()
for group in groups:
	pyplot.subplot(len(groups),1,i)
	pyplot.plot(values[:,group])
	pyplot.title(dataset.columns[group],y=0.5,loc='left')
	i += 1
pyplot.show()

In [ ]:
groups = range(8,14)
i = 1
pyplot.figure()
for group in groups:
	pyplot.subplot(len(groups),1,i)
	pyplot.plot(values[:,group])
	pyplot.title(dataset.columns[group],y=0.5,loc='left')
	i += 1
pyplot.show()

In [ ]:
# fix for 'package not found' when installing in Anaconda environment
import pip
pip.main(['install', 'xgboost'])

In [ ]:
column_to_predict = 'terapia_intensiva'

n_days = 250
dataset_reduced = dataset.iloc[-n_days:,:]

y = dataset_reduced[column_to_predict]
X = dataset_reduced.drop([column_to_predict],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

n_predictions = 20

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    shuffle=False,
    test_size=n_predictions
    )
regressor = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000
    )

regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, y_pred)

In [ ]:
pyplot.plot(y_test.values, label = "real")
pyplot.plot(y_pred, label = "predicted")
pyplot.show()